In [25]:
import pandas as pd
import numpy as np
from rmsle import ag_rmsle_clamped_scorer

train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')
label = 'Premium Amount'

# **AutoGluon**

In [26]:
from autogluon.tabular import TabularPredictor
autogluon_model = TabularPredictor(
    label=label,
    eval_metric='rmse' #ag_rmsle_clamped_scorer
)
autogluon_model.fit(
    train,
    presets='best_quality',
    time_limit=3*60*60,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241228_163852"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.1.0: Thu Oct 10 21:03:15 PDT 2024; root:xnu-11215.41.3~2/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       5.26 GB / 16.00 GB (32.9%)
Disk Space Avail:   222.96 GB / 460.43 GB (48.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_s

(_ray_fit pid=77602) [1000]	valid_set's rmse: 837.77
(_ray_fit pid=77604) [1000]	valid_set's rmse: 837.755 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=77602) [2000]	valid_set's rmse: 837.311 [repeated 2x across cluster]
(_ray_fit pid=77604) [2000]	valid_set's rmse: 837.343 [repeated 3x across cluster]
(_ray_fit pid=77601) [3000]	valid_set's rmse: 835.815
(_ray_fit pid=77602) [3000]	valid_set's rmse: 837.218
(_ray_fit pid=77602) [4000]	valid_set's rmse: 837.269 [repeated 2x across cluster]
(_ray_fit pid=77633) [1000]	valid_set's rmse: 838.016 [repeated 2x across cluster]
(_ray_fit pid=77639) [1000]	valid_set's rmse: 838.978
(_ray_fit pid=77633) [2000]	valid_set's rmse: 837.641
(_ray_fit pid=77650) [1000]	valid_set's rmse: 836.729 [repeated 2x across cluster]
(_ray_fit pid=77633)

(_dystack pid=77568) 	-837.9725	 = Validation score   (-root_mean_squared_error)
(_dystack pid=77568) 	177.89s	 = Training   runtime
(_dystack pid=77568) 	54.56s	 = Validation runtime
(_dystack pid=77568) Fitting model: LightGBM_BAG_L1 ... Training model for up to 1597.59s of the 2495.87s of remaining time.
(_dystack pid=77568) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 16.10% memory usage per fold, 64.41%/80.00% total).
(_dystack pid=77568) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=2, gpus=0, memory=16.10%)


(_ray_fit pid=77668) [1000]	valid_set's rmse: 833.463


(_dystack pid=77568) 	-835.3215	 = Validation score   (-root_mean_squared_error)
(_dystack pid=77568) 	40.88s	 = Training   runtime
(_dystack pid=77568) 	10.86s	 = Validation runtime
(_dystack pid=77568) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 1553.94s of the 2452.22s of remaining time.
(_dystack pid=77568) 	-836.3109	 = Validation score   (-root_mean_squared_error)
(_dystack pid=77568) 	738.98s	 = Training   runtime
(_dystack pid=77568) 	21.64s	 = Validation runtime
(_dystack pid=77568) Fitting model: CatBoost_BAG_L1 ... Training model for up to 792.87s of the 1691.15s of remaining time.
(_dystack pid=77568) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 15.44% memory usage per fold, 61.75%/80.00% total).
(_dystack pid=77568) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=2, gpus=0, memory=15.44%)
(_ray_fit pid=77758) 	Ran out of time, early stoppin

# **MLJar**

In [27]:
from supervised.automl import AutoML

mljar_model = AutoML(
    eval_metric='rmse',
    mode='Compete',
    total_time_limit=3*60*60
)
mljar_model.fit(
    train.drop(columns=[label]),
    train[label]
)
#predictions = automl.predict(test[[label]])

Linear algorithm was disabled.
AutoML directory: AutoML_4
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'mix_encoding', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 859.993049 trained in 461.52 seconds
Disable stacking for split validation
* Step simple_algorithms will try to check up to 2 models
2_DecisionTree rmse 857.173619 trained in 473.27 seconds
3_DecisionTree rmse 857.173619 trained in 487.26 seconds
* Step default_algorithms will try to check up to 6 model

AutoML(eval_metric='rmse', mode='Compete', total_time_limit=10800)

(raylet) The node with node id: 41bb31ae2603fd078ce011738f23a47d81c5a7eddba38fdb6b817762 and address: 127.0.0.1 and node name: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.
(raylet) Raylet is terminated. Termination is unexpected. Possible reasons include: (1) SIGKILL by the user or system OOM killer, (2) Invalid memory access from Raylet causing SIGSEGV or SIGBUS, (3) Other termination signals. Last 20 lines of the Raylet logs:
    [state-dump] 
    [state-dump] 
    [2024-12-30 22:31:48,378 I 77564 6540362] (raylet) accessor.cc:762: Received notification for node, IsAlive = 0 node_id=41bb31ae2603fd078ce011738f23a47d81c5a7eddba38fdb6b817762
    [2024-12-30 22:31:48,393 C 77564 6540362] (raylet) node_manager.cc:1043: [Timeout] Exiting because this node manager has mistakenly been marked as dead by the

# **TPOT**

In [23]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
prep = enc.fit_transform(train.drop(columns=[label, 'Policy Start Date']))

In [ ]:
from tpot import TPOTRegressor

tpot_model = TPOTRegressor(
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    max_time_mins=3*60,
)
tpot_model.fit(
    prep,
    train[[label]].rename(columns={label: 'class'})
)
#print(tpot_model.score(X_test, y_test))
#tpot_model.export('tpot.py')

# **TO-DO: Train NN to ensemble AutoML models**

In [4]:
#predictions = predictor.predict(test)
#test[label] = np.maximum(predictions.to_list(), 0)
#test[label].to_csv('submission_5.csv')